# Steps
**Read file and encoding**
* Read the input file.
* Select unique characters or classes from full text as vocub.
* Encoding: Generate number for each classes or characters.
* Replace all text with numbers using generated integers-character mapping.

[**Create Sequence for RNN input**](#create_sequence)
* Convert the encoded(number replaced text) text as sequences
* Use the `batch_size` and `number_of_steps` for mini batch
* Create a total batch with mini batch, This will have K number of main batch

[**Initialization of TF**](#tf_initialize)
* Initialize or define `input`, `output` and `keep_prob` for RNN
* Initialize or define Single cell LSTM
* Initialize Multicell or Multilayer LSTM using a single
* Initialize Output and State of LSTM.

In [1]:
import time
from collections import namedtuple
import numpy as np
import tensorflow as tf

In [1]:
import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
tf.__version__

'1.5.0'

# Input Preparations

Read the file as text.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
print(text[100:200])

g was in confusion in the Oblonskys' house. The wife had
discovered that the husband was carrying on


Create unique characters from total text as **vocabulary**. This helps to decide the **number of classes** or characters in a specific text.

In [3]:
vocab = sorted(set(text))
print(vocab)

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Map letters with numbers. Because our Network understand only the numbers not the characters

In [4]:
vocab_to_int = {c: i for i, c in enumerate(vocab)}
print(vocab_to_int)

{'2': 17, '/': 14, 'Y': 53, '5': 20, '!': 2, 'c': 59, '&': 6, 'e': 61, 'I': 37, 'O': 43, 'H': 36, 'Q': 45, 'h': 64, 'A': 29, 'b': 58, '(': 8, 'X': 52, 'J': 38, 'z': 82, 'v': 78, 'n': 70, ' ': 1, "'": 7, '3': 18, '"': 3, 'R': 46, '.': 13, '*': 10, '\n': 0, '1': 16, '0': 15, 'T': 48, '8': 23, 'S': 47, 'r': 74, '7': 22, 'N': 42, 's': 75, ':': 25, 'Z': 54, 'M': 41, 'F': 34, '?': 27, '6': 21, 'x': 80, ')': 9, 'm': 69, 'G': 35, '`': 56, 'C': 31, 'd': 60, 'w': 79, 'W': 51, '4': 19, 'p': 72, 'i': 65, 'q': 73, 'f': 62, 'K': 39, 'E': 33, 'V': 50, 'y': 81, '9': 24, 'u': 77, 'D': 32, '_': 55, 'U': 49, ',': 11, 'a': 57, '@': 28, 'l': 68, ';': 26, 'j': 66, '-': 12, 'k': 67, '$': 4, '%': 5, 't': 76, 'L': 40, 'B': 30, 'o': 71, 'P': 44, 'g': 63}


Using the mapping convert all letters as numbers

In [5]:
# Map letter with numbers, Inverse of integer mapping
int_to_vocab = dict(enumerate(vocab))

encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
print(text[100:200])
print(encoded[100:200])

g was in confusion in the Oblonskys' house. The wife had
discovered that the husband was carrying on
[63  1 79 57 75  1 65 70  1 59 71 70 62 77 75 65 71 70  1 65 70  1 76 64 61
  1 43 58 68 71 70 75 67 81 75  7  1 64 71 77 75 61 13  1 48 64 61  1 79 65
 62 61  1 64 57 60  0 60 65 75 59 71 78 61 74 61 60  1 76 64 57 76  1 76 64
 61  1 64 77 75 58 57 70 60  1 79 57 75  1 59 57 74 74 81 65 70 63  1 71 70]


# Convert inputs as batch sequences <a class="anchor" id="create_sequence"></a>

In [7]:
batch_size = 64; n_steps = 50

Collect Characters per Sequence or mini Batch

In [9]:
chars_per_batch = batch_size * n_steps
print(chars_per_batch)

3200


Total Number of Batches, Once we devide the total chars per mini batch, We able to get the Batch size

In [10]:
n_batchers = len(encoded)//chars_per_batch
print(n_batchers)

620


Based on possible Batch size. Select charactors that can create **full mini batch**

In [11]:
new_encoded = encoded[: chars_per_batch * n_batchers]
print('old_text_length: {}, new_text_length: {}'.format(len(encoded), len(new_encoded)))

old_text_length: 1985223, new_text_length: 1984000


Reshape the batch according to mini batch size.

In [12]:
new_encoded = new_encoded.reshape((batch_size, -1))

Create mini batch with number of steps. In otherwords, How many steps we are going to look back to create RNN or LSTM.

The `yield` function behave as `return`, Then reason we use `yeild` is, This won't keep the data in memory. When ever we need this will get executed and return the values.

To create input and output, We move the output by 1 step. Where if the input in $i^{th}$ position, Then the output will be $(i+1)^{th}$ position.

In [13]:
def get_batch(new_encoded, n_steps):
    for n in range(0, new_encoded.shape[1], n_steps):
        x = new_encoded[:, n: n+ n_steps]
        y_temp = new_encoded[:, n+1:n+n_steps+1]
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:y_temp.shape[1]] = y_temp
        yield x, y

In [14]:
batch = get_batch(new_encoded, n_steps)

Get each value from `generator` Object by using `next` function.

In [15]:
x, y = next(batch)
print(x)
print(y)

[[31 64 57 ...,  1 77 70]
 [70  1 76 ..., 81  1 69]
 [70 71 78 ..., 59 57 68]
 ..., 
 [70 60  1 ..., 78 61 74]
 [11  0 71 ...,  1 57 70]
 [ 1 70 71 ..., 70 71 76]]
[[64 57 72 ..., 77 70 64]
 [ 1 76 71 ...,  1 69 65]
 [71 78 65 ..., 57 68  1]
 ..., 
 [60  1 75 ..., 61 74  1]
 [ 0 71 74 ..., 57 70 81]
 [70 71 79 ..., 71 76 64]]


# Creation of Tensorflow variables. <a class="anchor" id="tf_initialize"></a>
## Input Variables
Initialize `inputs`, `outputs` and `keep_prob` probability in hidden units (hidden neurons in LSTM cell). The size of the input and output is $batch\_size \times n\_step$. Because we are always going to provide the input as mini batch as well as tensors. Here `keep_prob` is a scaler and 0 dimentional tensor

In [16]:
inputs = tf.placeholder(tf.int32, shape=[batch_size, n_steps], name='inputs')
targets = tf.placeholder(tf.int32, shape=[batch_size, n_steps], name='targets')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [17]:
print(inputs)
print(targets)
print(keep_prob)

Tensor("inputs:0", shape=(64, 50), dtype=int32)
Tensor("targets:0", shape=(64, 50), dtype=int32)
Tensor("keep_prob:0", dtype=float32)


In [18]:
lstm_size = 128 # Number of hidden units in LSTM cell

1. Create a single LSTM cell using `tf.contrib.rnn.BasicLSTMCell(lstm_size)`. From single cell we can create a multi Layer LSTM cells using `MultiRNNCell` Function.
2. We need to Initialize the cell states with zero. Here the state is the value that pass to another state(from time `t` to time `t+1`). When we create the states with cells. That will automatically create for hidden LSTM cells as well. This helps to pass the information to next state to remember things. 

We have 2 hidden layers or 2 LSTM cells. We have used a function to generate 2 LSTM cells dynamically. Initially I have tried this with by multiplying list with a value. `[cell] * number_of_layers`. Because If we multiply the same cell then we are just reusing it. But we need to create a new cell. Therefore this has given a error while creating a RNN Layer with `tf.nn.dynamic_rnn`. Therefore created a function to generate each cell. Here we don't pass any data. We are just defineing the cells.

In [19]:
def create_single_cell(lstm_size, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    return drop

In [20]:
number_of_layers = 2

Stack the LSTM cell in to Mutliple layers

In [21]:
cells = tf.contrib.rnn.MultiRNNCell([create_single_cell(lstm_size, keep_prob) for _ in range(number_of_layers)])

Create initial state for each cell. The `cell.zero_state` function will create initial values for states. Since initial input size at time t is `Number of sequence` or `batch_size`. So each cell will be created with `batch_size` of initial state.

In [22]:
# values that pass to time-steps
initial_state = cells.zero_state(batch_size, tf.float32)

In [23]:
print(cells)
initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(64, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(64, 128) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState_1/BasicLSTMCellZeroState/zeros:0' shape=(64, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState_1/BasicLSTMCellZeroState/zeros_1:0' shape=(64, 128) dtype=float32>))

# Creation of RNN Network

The class of `one_hot` encoder is `vocub`: Unique characters from whole text. The number of classes helps to find the probability of each letter by creating `logits`, The logit is a vector for each character after applying softmax functions. Because when we are passing a single character we will pass a vector, where the specific character will marked as 1 amoung a vector of classes. And from RNN output the character comes with probability distribution from softmax function. This will be input for each LSTM cell as vector.

In [24]:
print('Classes or charactors')
print(vocab)

Classes or charactors
['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Once we pass through `one_hot`, It show a 3D result of input. Where
* 64: `batch_size`
* 50:`number_of_steps`
* 83: `classes`

Because the `tf.one_hot` converts the each character vector size of 83. We normally we call this process as one hot encoding.

In [25]:
num_classes = len(vocab)
print('number of classes: ' + str(num_classes))
x_one_hot = tf.one_hot(inputs, num_classes)
print(x_one_hot)

number of classes: 83
Tensor("one_hot:0", shape=(64, 50, 83), dtype=float32)


Create a RNN architecture by combining all initialized cell or LSTM Multilayer

* **Output**: This is the actual output from LSTM cell.
* **State**: This is a output from LSTM cell that pass to next time step.

In [26]:
outputs, state = tf.nn.dynamic_rnn(cells, x_one_hot, initial_state=initial_state)

In [27]:
final_state= state

Now we have create a RNN network with 2 hidden layer or 2 LSTM cells. And each character input has been encoded with One Hot Encoding.

# Reshape Output

<img src="assets/charRNN.png" width=500px>

We have $N$ steps in mini match. 
- So we are going to get $N$ outputs from our RNN. 

At the same time we are passing multiple sequence $M$ in parallel. 
- So we get another set of output for each sequence. So the Total output will be $M \times N$

LSTM also has $L$ number of hidden units.
- So the total output from LSTM is 3 dimentional. Which is $M \times N \times L$

In [28]:
outputs

<tf.Tensor 'rnn/transpose:0' shape=(64, 50, 128) dtype=float32>

Actually we will get LSTM outputs as `list`. So the list need to concatenated to create an array.

In [31]:
seq_output = tf.concat(outputs, axis=1)
seq_output

<tf.Tensor 'concat_2:0' shape=(64, 50, 128) dtype=float32>

All the outputs from LSTM are going through the **same sigmoid** layer with the **same weights**. Therefore we can reshape this output from LSTM in to 2D tensor with the shape $(M * N) \times L$. That is, 
* Each row output for **each charactor**. Where the total charactors are $M*N$. In other words **each row** is **one step** and **one sequence**. Or Every row is one output from one LSTM cell.
* $M*N$ rows with $L$ columns, Where $charactors\ per\ mini\ batch = M * N$

Then the array need to reshaped as we have discussed above. Where each row for each charactor with $L$ number of columns.

Actually this reshape will helps with Softmax function matrix multiplication

In [43]:
output_reshape = tf.reshape(seq_output, [-1, lstm_size])

In [44]:
output_reshape

<tf.Tensor 'Reshape:0' shape=(3200, 128) dtype=float32>

Once we have reshaped. Then we can create the weights and bias for the softmax layer. Then we will do the matrix multiplication and adding bias to get the logits. Then we will pass the logits through the softmax function.

The weights and bias need to be created with ```python tf.variable_scope('name_for_softmax')```. Because RNN cell also have weights and bias with the default. So If we define softmax weight and bias without `tf.variable_scope`. Then this will take the default name as RNN. And that will throw an error. To avoid the error we pass a different name for the softmax function weights and the bias.

<img src="assets/softmaxcell.png" width=500px>

The shape of the weights will be Number of hidden units in a LSTM cell and the size of class/unique characters.

In [45]:
with tf.variable_scope('softmax_variables'):
    softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(num_classes))

In [46]:
print(softmax_w)
print(softmax_b)

<tf.Variable 'softmax_variables_1/Variable:0' shape=(128, 83) dtype=float32_ref>
<tf.Variable 'softmax_variables_1/Variable_1:0' shape=(83,) dtype=float32_ref>


In [47]:
prediction = tf.matmul(output_reshape, softmax_w) + softmax_b

In [48]:
prediction

<tf.Tensor 'add:0' shape=(3200, 83) dtype=float32>

Now we have logits for each charactor in mini batch.

Apply the softmax function for output logits from each LSTM

In [49]:
logits = tf.nn.softmax(prediction, name='predictions')

In [50]:
logits

<tf.Tensor 'predictions:0' shape=(3200, 83) dtype=float32>

# Training loss

Here we will be using `cross entrophy` loss to find the error loss. The tensorflow function support with `tf.nn.softmax_cross_entropy_with_logits` then find the mean to get the loss actual loss.

The **targets need to be reshaped in to 2D tensor** as we have done with RNN outputs. Basically the logits and the targets need to be in the same shape. logits is the one that come throgh softmax function.

In [51]:
targets

<tf.Tensor 'targets:0' shape=(64, 50) dtype=int32>

In [52]:
y_one_hot = tf.one_hot(targets, num_classes)

In [53]:
y_one_hot

<tf.Tensor 'one_hot_2:0' shape=(64, 50, 83) dtype=float32>

In [54]:
y_out_reshape = tf.reshape(y_one_hot, logits.get_shape())

In [55]:
y_out_reshape

<tf.Tensor 'Reshape_1:0' shape=(3200, 83) dtype=float32>

Calculate the loss: Return a probability distribution for each character.

In [56]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_out_reshape)

In [57]:
loss

<tf.Tensor 'Reshape_4:0' shape=(3200,) dtype=float32>

In [58]:
loss = tf.reduce_mean(loss)

In [59]:
loss

<tf.Tensor 'Mean:0' shape=() dtype=float32>

# Optimizer

Here we build the optimizer. Normal RNNs has issues with gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

Map all the variables that we have assigned with this network. And this is handled by the Graph of RNN. To reset this mapping or graph we need to use `tf.reset_default_graph()`

In [60]:
grad_clip = 5;
learning_rate = 0.01

In [63]:
# Optimizer for training, using gradient clipping to control exploding gradients
tvars = tf.trainable_variables()

In [64]:
tvars

[<tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(211, 512) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(256, 512) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'softmax_variables/Variable:0' shape=(128, 83) dtype=float32_ref>,
 <tf.Variable 'softmax_variables/Variable_1:0' shape=(83,) dtype=float32_ref>,
 <tf.Variable 'softmax_variables_1/Variable:0' shape=(128, 83) dtype=float32_ref>,
 <tf.Variable 'softmax_variables_1/Variable_1:0' shape=(83,) dtype=float32_ref>]

In [48]:
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)

In [49]:
grads

[<tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_0:0' shape=(211, 512) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(512,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_2:0' shape=(256, 512) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_3:0' shape=(512,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_4:0' shape=(128, 83) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_5:0' shape=(83,) dtype=float32>]

In [50]:
train_op = tf.train.AdamOptimizer(learning_rate)

In [51]:
train_op

In [52]:
optimizer = train_op.apply_gradients(zip(grads, tvars))

In [53]:
optimizer

<tf.Operation 'Adam' type=NoOp>

# Build the network

To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN.